In [45]:
import pandas as pd
import numpy as np
from geopy.distance import distance as geo_dist

In [2]:
# Extract list of stop from metadata file.
with open("data/BFKOORD_GEO") as file:
    metadata = file.readlines()
    
metadata_cleaned = [line.split("%") for line in metadata]
metadata_cleaned = [[line[0].split(), line[1][1:-1]] for line in  metadata_cleaned]
metadata_cleaned = [[line[0][0], line[0][1], line[0][2], line[0][3], line[1]] for line in metadata_cleaned]

metadata_df = pd.DataFrame(metadata_cleaned, columns=["StationID", "Longitude", "Latitude", "Height", "Remark"])

metadata_df["Longitude"] = pd.to_numeric(metadata_df["Longitude"])
metadata_df["Latitude"] = pd.to_numeric(metadata_df["Latitude"])

In [73]:
# Retrieve stops near Zurich main station
zurichHB = metadata_df[metadata_df.Remark == "Zürich HB"]

metadata_df["dist"] = metadata_df.apply(lambda x: geo_dist((zurichHB.Latitude.values[0], zurichHB.Longitude.values[0]),
                                                            (x.Latitude, x.Longitude)).km, axis=1)
zurich_stops_df = metadata_df[metadata_df["dist"] < 10]
zurich_stops_df.head()

,StationID,Longitude,Latitude,Height,Remark,dist,dist1
74,0000176,8.521961,47.351679,0,Zimmerberg-Basistunnel,3.251969,3.251969131789274 km
2084,8502220,8.434713,47.390882,442,Urdorf,8.088858,8.088858133324928 km
2085,8502221,8.437543,47.357432,488,Birmensdorf ZH,8.089105,8.089104631368896 km
2086,8502222,8.468175,47.325896,528,Bonstetten-Wettswil,7.961914,7.961913838714972 km
2092,8502229,8.430330,47.380971,456,Urdorf Weihermatt,8.302117,8.302117210253517 km


In [41]:
import os

zurich_stops_lower_name = zurich_stops_df.Remark.str.lower()
for d in os.listdir("data/raw"):
    for f in os.listdir(os.path.join("data/raw", d)):
        print(f, end="\r")
        data_df = pd.read_csv(os.path.join("data/raw", os.path.join(d, f)), sep=";").dropna(subset = ["HALTESTELLEN_NAME"])
        filtered_data = data_df[data_df.HALTESTELLEN_NAME.str.lower().isin(zurich_stops_lower_name)]    
        filtered_data.to_csv(os.path.join("data/filtered", os.path.join(d, f)), sep=";", index=False)

/home/student/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
